<a href="https://colab.research.google.com/github/marek-bardonski/airev-advanced-workshops/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip3 install wget
print ("Hello world!")
import wget
from zipfile import ZipFile
import os
import cudf

Hello world!


In [0]:
# Raw datasets are samples from WebHose.io 
print('Beginning dataset download with wget module')

url = 'https://bardonski.pl/chineese.zip'
wget.download(url)
url = 'https://bardonski.pl/arabic.zip'
wget.download(url)

In [0]:
zip = ZipFile('chineese.zip')
zip.extractall()

!mkdir chineese
zip = ZipFile('630_webhose-2016-10_20170904084325.zip')
zip.extractall('chineese')

630_webhose-2016-10_20170904084325.zip	chineese      sample_data
arabic.zip				chineese.zip


In [0]:
zip = ZipFile('arabic.zip')
zip.extractall()

!mkdir arabic

In [0]:
zip = ZipFile('627_webhose-2016-10_20170904083346.zip')
zip.extractall('arabic')


In [11]:
#Expected 236384 arabic articles
#Expected 316004 chineese articles
!ls arabic -l | wc -l
!ls chineese -l | wc -l


236384
316004


In [12]:
def get_txt_lines(data_dir):
    """
        Read text lines from gutenberg tests
        returns (text_ls,fname_ls) where 
        text_ls = input_text_lines and fname_ls = list of file names
    """
    text_ls = []
    fname_ls = []
    for fn in os.listdir(data_dir):
        full_fn = os.path.join(data_dir,fn)
        with open(full_fn,encoding="utf-8",errors="ignore") as f:
            content = f.readlines()
            content = get_non_empty_lines(content)
            text_ls += content
            ### dont add .txt to the file
            fname_ls += [fn[:-4]]*len(content)
    
    return text_ls, fname_ls    
    
print("File Read Time:")
%time txt_ls,fname_ls = get_txt_lines('arabic')
df = cudf.DataFrame()

print("\nCUDF  Creation Time:")
%time df['text'] = nvstrings.to_device(txt_ls)

File Read Time:


NameError: ignored